In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import rasterio

# Notebook to generate head obs from DEC wells

### Read in DEC well shapefiles (from SB folder - these have been added to SB) and basin boundary for clipping

In [ ]:
bound_file = '../sciencebase/Shapefiles/Extents/Model_Extent_HUC12.shp'
dec_wells_file = '../sciencebase/Shapefiles/Wells/DEC_wells_ALL_NY.shp'
dec_water_wells_file = '../sciencebase/Shapefiles/Wells/waterwells.shp'

In [ ]:
bound = gp.read_file(bound_file)
bound.crs

In [ ]:
# this is a BIG shapefile! 

dec_wells = gp.read_file(dec_wells_file)
dec_wells.crs

In [ ]:
water_wells = gp.read_file(dec_water_wells_file)
water_wells.crs

### Reproject to model CRS: EPSG 5070

In [ ]:
bound = bound.to_crs(epsg=5070)
dec_wells = dec_wells.to_crs(epsg=5070)
water_wells = water_wells.to_crs(epsg=5070)

### Clip wells to model boundary (and plot to check)

In [ ]:
dec_wells_clip = gp.clip(dec_wells, bound)
water_wells_clip = gp.clip(water_wells, bound)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
bound.plot(ax=ax, edgecolor='k', facecolor='none')
dec_wells_clip.plot(ax=ax)
plt.title('DEC wells from provided by NYWSC on 7/16/2020')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
bound.plot(ax=ax, edgecolor='k', facecolor='none')
water_wells_clip.plot(ax=ax, c='red')
plt.title('DEC wells from water wells database')
plt.show()

## Combine the the two DEC well dataframes and work up into an obs file

In [ ]:
water_wells_clip['group'] = 'dec_water_wells'
water_wells_clip['obsnme'] = water_wells_clip['DEC_Well_N'] #  make common obsname

In [ ]:
dec_wells_clip['group'] = 'dec_from_nywsc'
dec_wells_clip['obsnme'] = dec_wells_clip['WellNO'] #  make common obsname

In [ ]:
dec_wells_all = dec_wells_clip.append(water_wells_clip, ignore_index=True)
dec_wells_all

In [ ]:
dec_wells_all_working = dec_wells_all.copy()

#  remove all wells with null GW_Depth
dec_wells_all_working = dec_wells_all_working.loc[dec_wells_all_working.GW_Depth.notnull()]

#  assume that zeros are null? Maybe not always the case but hard to know the zeros are real
dec_wells_all_working = dec_wells_all_working.loc[dec_wells_all_working.GW_Depth.astype(float) != 0]

#  finally, drop the 55 duplicates in 'dec_water_wells' 
dec_wells_all_working = dec_wells_all_working.drop_duplicates(subset=['obsnme'], keep='first') 

Land surface elevations in the DEC wells table look pretty unreliable. Want to get reasonable land surface elevations. Extract pixel values from land surface elevation raster at the location of each well. This probably improves land surface elevation values, from which we can calculate better groundwater elevations from the supplied depth to water values... assuming those depth to water values are at all realiable.

In [ ]:
def raster_point_value(input_raster, gdf):
    """
    Extracts raster pixel value from the well/point locations listed in a geodataframe.

    Properties
    ------------------------------
    input_raster: str, path to categorical landuse raster file for analysis
    gdf: geopandas geodataframe, contains of geometry of wells/points

    """

    #  extract raster crs
    with rasterio.open(input_raster) as src:
        raster_crs = src.crs

    #  transform geodataframe to raster crs
    gdf_reproj = gdf.to_crs(raster_crs)

    #  get a list of coordinate tuples for each row in a geopandas dataframe that contains point
    x_coords = gdf_reproj['geometry'].x.to_list()
    y_coords = gdf_reproj['geometry'].y.to_list()
    coords = list(zip(x_coords, y_coords))

    #  extract raster cell values at well/point coordinates
    with rasterio.open(input_raster) as src:
        point_values = list(np.concatenate([v for v in src.sample(coords)], axis=0))

    return point_values

In [ ]:
input_raster = '../sciencebase/Shapefiles/top_50m_from_lidar.tif'

#  use function above to get list of well elevations (m)
well_ls_elevs_m = raster_point_value(input_raster, dec_wells_all_working)

#  add computed well elevations as column in DEC wells df
dec_wells_all_working['ls_elev_m'] = well_ls_elevs_m

#  compute groundwater elevations in meters, using DEC supplied DTW in ft and computed LSE values in meters
dec_wells_all_working['gw_elev_m'] = dec_wells_all_working['ls_elev_m'] - (dec_wells_all_working['GW_Depth'].astype(float) * 0.3048)

In [ ]:
#  add "x" and "y" columns in CRS of model for mfsetup

dec_wells_all_working['x'] = dec_wells_all_working.geometry.x
dec_wells_all_working['y'] = dec_wells_all_working.geometry.y

In [ ]:
dec_wells_all_working['group'].unique().tolist()

**Looks like in the end we clipped out all of the DEC water wells and are only using the DEC wells supplied by NYWSC**

In [ ]:
dec_wells_all_working.to_csv('../obs_data/NY_DEC_GW_sites.csv')